In [121]:
import nltk
import sys
import pathlib
import os
import json
#from stemming.porter2 import stem

# Load Dataset

In [75]:
data_folder = str(pathlib.Path(os.path.abspath('')).parents[1])+'/data/'

In [76]:
source_texts = data_folder+"test_doc.txt"

data = []
# count each line individually
with open(source_texts, 'r') as file:
    # the indx of that line and the contents of that line
    for (indx, line) in enumerate(file):
        data.append(line)

In [77]:
# remove new line from the list and \n from the sentence
data = [d.replace('\n', '') for d in data if d != '\n']

In [78]:
print(len(data))

44


# Normalization and Tokenization

Normalization means

changing to the same form like to lower case

changing some shortform into their original word

e.g is't --> is not
   
Tokenize means just changin a given sentence into a list of independent tokens or words as much as possible.

but the greatest issue with Tokenization is its really difficult to tokenize into independent tokens.

because of name, country name e.g Addis Ababa becomes two tokens but should have one token,

In [80]:
#Adding another StopWords because there are only 179 english words in nltk
stopwords = nltk.corpus.stopwords.words('english')
other_stopwords = []
with open(data_folder+'stopwords.txt', 'r') as file:
    for st in file:
        other_stopwords.append(st)

other_stopwords = [x.replace('\n', '') for x in other_stopwords]
stopwords.extend(other_stopwords)

In [89]:
# difference with nltk stopwords and the updated stopword
#print(len(nltk.corpus.stopwords.words('english')))
#print(len(stopwords))

In [123]:
cleaned_file_dict = {}
for line in data:
    line = line.lower()
    # here for tokenization i used nltk.word_tokenize
    # TOKENIZATION
    tokens = nltk.word_tokenize(line) 
    #print(tokens)
    
    # now Cleaning
    clean_tokens = cleaning(tokens)
    
    #print(clean_tokens)
    final_tok = []
    for tok in clean_tokens:
        tok = str(tok).rstrip(' ')
        if(len(str(tok).split(' ')) > 1):
            # TOKENIZATION
            token = nltk.word_tokenize(tok)
            cl_tok = cleaning(token)
            for tk in cl_tok:
                final_tok.append(tk)
        else:
            final_tok.append(tok)
    
    # Now Removing STOPWORDS
    filtered_words = [w for w in final_tok if not w in stopwords]
    
    # To check the difference with the filtered words
    #print(final_tok)
    #print(filtered_words)
   
    # Now LEMMATIZATION
    lemmatizer = nltk.WordNetLemmatizer()
    w_net_lem = []
    for i in range(len(filtered_words)):
        w_net_lem.append(lemmatizer.lemmatize(filtered_words[i]))
     
    # Now STEMMING over the Lemmatized word
    po_stemmer = nltk.stem.PorterStemmer()
    
    #stemmed = [stem(w) for w in w_net_lem]
    stemmed = [po_stemmer.stem(w) for w in w_net_lem]
    
    #print(w_net_lem)
    #print(stemmed)
    
    # now save to file
    # connect each word in the list of stem to make it value for the loaded file as index in dictiona
    temp = ''
    for i in stemmed:
        temp = temp + i + ' '
    
    #print(temp)
    cleaned_file_dict[line] = temp
    
    with open(data_folder+'cleaned.json', 'w') as f:
        json.dump(cleaned_file_dict, f, indent=4)

# Cleaning


removing punctuation and unwanted character

including from the list of tokens and inside the token itself

e.g ['dr.', ',', 'son-in-law']

removing ',' from the list

changing 'dr.' into 'dr'

changin 'son-in-law' into 'son in law'

Note in Cleaning or Removing texts what we need to recall is

The Preprocessing depends on the Task we are doing.

so I assume that this preprocessing is for the Topic Analysis Task. or also for Search Engine

e.g if its Machine Translation or Text to Speech there may be No words that we remove or clean

In [6]:
import string

# characters that are not found in string.punctuation
punctuation = ['…','•','”','→','↑','“','‘','’','—','£','€','$']

def cleaning(tokens):
    cleaned_token = []
    
    # check for each token in each list of tokens(sentence or data)
    for token in tokens:
        # if the token has the punctuation.
        if ((token_has_punct(token))):
            # identify which character it is
            chrs = find_punct(token) 
            new_token = token
            for cr in chrs:
                new_token = str(new_token.replace(cr,' ')) if cr == '-' else str(new_token).replace(cr, '')
            
            # if the token is number or not an english word pass over it else append to the list
            if (is_float(new_token) or isNotEnglish(new_token)):
                pass
            else:
                cleaned_token.append(new_token) 
        # if token has not punctuation and is not an english word pass over it
        elif(isNotEnglish(token)):
            pass
        
        # if token has not puncuation and is an english word append to the list
        else:
            cleaned_token.append(token)

    return cleaned_token


def isNotEnglish(token):
    # check it by decoding the asii code of the character in token, 
    # if its no an english letter Error will raised
    try:
        token.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return True
    else:
        return False


def token_has_punct(token):
    # check each character in token
    for cr in token: 
        if ((cr in string.punctuation) or cr in punctuation): return True 
        else: pass
    return False

def find_punct(token):
    crs = []
    for cr in token: crs.append(cr) if ((cr in string.punctuation) or (cr in punctuation)) else '' 
    return crs

def is_float(x):
    try: 
        float(x) 
        return True
    except: 
        return False

# Lemmatization

For Lemmatization I used the Wordnet, which is a large, freely and publicly available lexical database for the English language

and aiming to establish structured semantic relationships between words.

It offers lemmatization capabilities as well is one of the earliest and most commonly used lemmatizer.


NLTK offers an interface to Wordnet but first we have to download it

In [102]:
#nltk.download('wordnet')

# Stemming

There are different algorithim for Stemming, like

PorterStemmer in the NLTK

Stemming 

so here I have used the PorterStemmer in the NLTK

eventhough they have a little difference with Stemming
